---
layout: post
title: Motivational Quote API
description: API that has Motivational Quotes that Users and Upvote and Downvote.
toc: false
comments: false
type: hacks
courses: {"compsci": {week: 11} }
---

### Code for API

In [ ]:
from flask import Blueprint, jsonify  # jsonify creates an endpoint response object
from flask_restful import Api, Resource # used for REST API building
import requests  # used for testing 
import random

from model.jokes import *

joke_api = Blueprint('joke_api', __name__,
                   url_prefix='/api/jokes')

# API generator https://flask-restful.readthedocs.io/en/latest/api.html#id1
api = Api(joke_api)

class JokesAPI:
    # not implemented
    class _Create(Resource):
        def post(self, joke):
            pass
            
    # getJokes()
    class _Read(Resource):
        def get(self):
            return jsonify(getJokes())

    # getJoke(id)
    class _ReadID(Resource):
        def get(self, id):
            return jsonify(getJoke(id))

    # getRandomJoke()
    class _ReadRandom(Resource):
        def get(self):
            return jsonify(getRandomJoke())
    
    # getRandomJoke()
    class _ReadCount(Resource):
        def get(self):
            count = countJokes()
            countMsg = {'count': count}
            return jsonify(countMsg)

    # put method: addJokeHaHa
    class _UpdateLike(Resource):
        def put(self, id):
            addJokeHaHa(id)
            return jsonify(getJoke(id))

    # put method: addJokeBooHoo
    class _UpdateJeer(Resource):
        def put(self, id):
            addJokeBooHoo(id)
            return jsonify(getJoke(id))

    # building RESTapi resources/interfaces, these routes are added to Web Server
    api.add_resource(_Create, '/create/<string:joke>')
    api.add_resource(_Read, '/')
    api.add_resource(_ReadID, '/<int:id>')
    api.add_resource(_ReadRandom, '/random')
    api.add_resource(_ReadCount, '/count')
    api.add_resource(_UpdateLike, '/like/<int:id>')
    api.add_resource(_UpdateJeer, '/jeer/<int:id>')
    
if __name__ == "__main__": 
    # server = "http://127.0.0.1:5000" # run local
    server = 'https://flask.nighthawkcodingsociety.com' # run from web
    url = server + "/api/jokes"
    responses = []  # responses list

    # get count of jokes on server
    count_response = requests.get(url+"/count")
    count_json = count_response.json()
    count = count_json['count']

    # update likes/dislikes test sequence
    num = str(random.randint(0, count-1)) # test a random record
    responses.append(
        requests.get(url+"/"+num)  # read joke by id
        ) 
    responses.append(
        requests.put(url+"/like/"+num) # add to like count
        ) 
    responses.append(
        requests.put(url+"/jeer/"+num) # add to jeer count
        ) 

    # obtain a random joke
    responses.append(
        requests.get(url+"/random")  # read a random joke
        ) 

    # cycle through responses
    for response in responses:
        print(response)
        try:
            print(response.json())
        except:
            print("unknown error")

### Code for Model

In [ ]:
import random

quotes_data = []
quote_list = [
    "The only way to do great work is to love what you do.",
    "Believe you can and you're halfway there.",
    "Success is not final; failure is not fatal: It is the courage to continue that counts.",
    "The future depends on what you do today.",
    "You are never too old to set another goal or to dream a new dream.",
    "In the middle of every difficulty lies opportunity.",
    "The only limit to our realization of tomorrow will be our doubts of today.",
    "Success is walking from failure to failure with no loss of enthusiasm.",
    "Your time is limited, don't waste it living someone else's life.",
    "The best way to predict the future is to create it.",
    "The master has failed more times than the student has even attempted"
    "Fall seven times and rise eight"
    "Yesterday is history, tomorrow is a mystery, but today is a gift. That is why is called the present"
]

# Initialize quotes
def initQuotes():
    # Setup quotes into a dictionary with id, text, upvotes, downvotes
    item_id = 0
    for item in quote_list:
        quotes_data.append({"id": item_id, "text": item, "upvotes": 0, "downvotes": 0})
        item_id += 1
    # Prime some upvotes
    for i in range(5):
        id = getRandomQuote()['id']
        upvoteQuote(id)
    # Prime some downvotes
    for i in range(3):
        id = getRandomQuote()['id']
        downvoteQuote(id)

# Return all quotes from quotes_data
def getQuotes():
    return quotes_data

# Quote getter
def getQuote(id):
    return quotes_data[id]

# Return random quote from quotes_data
def getRandomQuote():
    return random.choice(quotes_data)

# Most liked quote
def favoriteQuote():
    best = 0
    bestID = -1
    for quote in getQuotes():
        if quote['upvotes'] > best:
            best = quote['upvotes']
            bestID = quote['id']
    return quotes_data[bestID]

# Most downvoted quote
def jeeredQuote():
    worst = 0
    worstID = -1
    for quote in getQuotes():
        if quote['downvotes'] > worst:
            worst = quote['downvotes']
            worstID = quote['id']
    return quotes_data[worstID]

# Add an upvote for the requested id
def upvoteQuote(id):
    quotes_data[id]['upvotes'] = quotes_data[id]['upvotes'] + 1
    return quotes_data[id]['upvotes']

# Add a downvote for the requested id
def downvoteQuote(id):
    quotes_data[id]['downvotes'] = quotes_data[id]['downvotes'] + 1
    return quotes_data[id]['downvotes']

# Pretty Print quote
def printQuote(quote):
    print(quote['id'], quote['text'], "\n", "Upvotes:", quote['upvotes'], "\n", "Downvotes:", quote['downvotes'], "\n")

# Number of quotes
def countQuotes():
    return len(quotes_data)

# Test Quote Model
if __name__ == "__main__":
    initQuotes()  # Initialize quotes

    # Most liked and most downvoted quotes
    best = favoriteQuote()
    print("Most liked", best['upvotes'])
    printQuote(best)
    worst = jeeredQuote()
    print("Most downvoted", worst['downvotes'])
    printQuote(worst)

    # Random quote
    print("Random quote")
    printQuote(getRandomQuote())

    # Count of Quotes
    print("Quotes Count: " + str(countQuotes()))
